[Cython]: http://cython.org/
[docs]: http://docs.cython.org/en/latest/src/quickstart/index.html
[wiki]: https://github.com/cython/cython/wiki

[tutorial]: http://conference.scipy.org/proceedings/SciPy2009/paper_1/full_text.pdf
[numerical calculations]: http://conference.scipy.org/proceedings/SciPy2009/paper_2/full_text.pdf
[tinyr]: https://code.google.com/archive/p/tinyr/


# Cython

We now will try [Cython] as a solution to speedup our RBush ([docs], [wiki]).
Cython is a superset of Python, where special syntax can be used to call C-level functions and to compile the \*ython code as C.
Cython claims that by using the cython compiler engine even pure-python code can benefit (even if slightly) from (use of C) optimizations.

Two documents should help us start this journey, the Cython [tutorial] by *S. Behnel, R.W. Bradshaw & D.S. Seljebotn* and a benchmark on [numerical calculations] done by *Dag Sverre Seljebotn*.
We also take [tinyr], an implementation of a r-tree, as our starting point.

If you're using Anaconda, Cython is available through the default channel for install.

[Sage]: http://www.sagemath.org/

## Cython tutorial

From the tutorial we learn the following.

Cython code can be compiled using:
* `setup.py` setup (what we will eventually use);
* `pyximport` to call cython' `pyx` files as if they were `py` modules and have compilation done in the background;
* pre-compile the code with `cython` command-line utility (most for debugging/tests);
* by using [Sage] notebooks, which allows Cython code inline (most for experimentation).



## `tinyr`

From `tinyr.pyc` we have the following:

* function definition:
```python
cdef tuple array_to_tuple_interleaved(Dfloat *coords):
    return tuple([ coords[i] for i in range(4)])
```
    
* variable declaration:
```python
cdef inline void common_boundaries(list records, Dfloat *target):
    cdef:
        Dfloat *coords
        _Record r
```

* forward declaration:
```python
cdef class _Record
```

* tyde definition:
```python
ctypedef double Dfloat
```

* declare a class:
```python
cdef class _Record:
    cdef:
        Dfloat coords[4]

    cdef inline bint overlaps(self, Dfloat *rect):
        return self.coords[0] < rect[2] and self.coords[2] > rect[0] and self.coords[1] < rect[3] and self.coords[3] > rect[1]
    
    cdef inline void copy_coords_to(self, Dfloat *coords):
        for i in range(4):
            coords[i] = self.coords[i]
    
    cdef _Record copy(self, _Node newparent):
        raise NotImplemented
```



# Profiling

We're now at py-rbush commit #`6b48a144f0d602c90cc32d9c82771a131242fedc`, cython functions to `insert` and `search` have been adapted using pieces of `tinyr` and is now time to profile those functions.

The new version using `cython` is doing much better then our previous attempt with `numba` (whenever possible\*), but it still doing worse (~10x) than the brute-force\*\* method -- this one using all numba. Since it is a brute-force, and the purpose of using a tree structure is to reduced (logarithmically) the time consumed, we have to figure out where/what is going on and fix it until we get a substantial improvement (~100x?) to a typical use case (1M items).

To figure that out, we have to profile the functions in use and, probably, eventually we'll go down to line profiling.

\*: we couldn't use numba (version 0.36) everywhere in `rbush` because the library still has some limitations: it does not accept lists with non-homogeneous (*i.e*, different sizes) items, and when items were abstracted in (numba) classes the performance gain was quite low.

\*\*: the brute-force method is to search the items (boxes) linearly, one-by-one in our entire dataset.

## python --> cython

Python code is typically profiled using the stdlib `profile` or `cProfile` [profilers]. We can also easily inspect the performance of our code line-by-line using the [line_profiler] package.

Cython profiling is pretty close to a pure python code, we just have to add some pragmas to it according to how/what we want to inspect; their [profiling tutorial] gives us a nice first-steps guide.
Some googling on the subject inform us that the [line_profiler may be used with cython][so_lp].

So, here is the summary (so far, from what I got):
* we can use `cprofile` to profile an entire (`pyx`) module;
* we exclude some functions out of interest by decorating it accordingly;
* we can use line-tracing (with cython/cprofile);
* we can annotate the line-tracing with the `coverage` package;
* we can use Kern's `line_profiler` to have the timings per-line.

[profilers]: https://docs.python.org/3/library/profile.html
[line_profiler]: https://github.com/rkern/line_profiler
[profiling tutorial]: http://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html
[so_lp]: https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line

## The simplest first

First step I will take is to simply enable the profiling using nothing more then the minimal settings.
This means we'll do as with a pure-python code: we use cProfile while running our test case.

The only addition is to include the pragma
```python
# cython: profile=True
```
to the top of the (`pyx`) module being profiled (file `core_funcs.pyx`).

In [1]:
import cProfile
import pstats

import pyximport
pyximport.install()

from rbush.data import generate_data_array
import rbush

def runme():
    data = generate_data_array(100000)
    r = rbush.RBush()
    r.load(data)
    dt = generate_data_array(1000)
    for i in range(len(dt)):
        _ = r.search(*tuple(dt[i]))


cProfile.runctx('runme()', globals(), locals(), 'profile.prof')

s = pstats.Stats('profile.prof')
s.strip_dirs().sort_stats('time').print_stats()

Wed Jan 24 20:44:16 2018    profile.prof

         78729423 function calls (70340609 primitive calls) in 26.272 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000   10.058    0.010   20.569    0.021 core_funcs.pyx:437(search)
8389814/1000    5.440    0.000    6.663    0.007 core_funcs.pyx:61(find_overlapping_leafs_recursive)
 49279867    5.069    0.000    5.069    0.000 core_funcs.pyx:96(overlaps)
   497971    1.595    0.000    2.302    0.000 core_funcs.pyx:31(choose_subtree_least_enlargement)
 11917518    0.729    0.000    0.729    0.000 core_funcs.pyx:508(area)
    20489    0.605    0.000    0.846    0.000 core_funcs.pyx:331(PickSeeds)
   138950    0.554    0.000    0.811    0.000 core_funcs.pyx:367(PickNext)
  5823958    0.495    0.000    0.495    0.000 core_funcs.pyx:511(common_boundaries)
        1    0.325    0.325   26.272   26.272 <ipython-input-1-9fcd12acd91f>:10(runme)
   100000    0.324    0.000    0.867  

Notice that we just profiled everything in our code.

If we want we can disable the functions out of our interest by decorating them with:
```python
cimport cython
@cython.profile(False)
def non_interesting_function():
    # ...
```

The functions we are not concerned about now (*i.e*, the ones used during `load/insert`) are `insert`, `insert_at_node`, `_AdjustTree`, `SplitNode`; by all means they are consuming just a fraction of the total time.
Anyway, for the sake of this exercise, if I disable their profiling and run it again we have the following:
```
Wed Jan 24 20:59:10 2018    profile.prof

         77651991 function calls (69320271 primitive calls) in 26.284 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000   10.055    0.010   20.502    0.021 core_funcs.pyx:446(search)
8332720/1000    5.458    0.000    6.668    0.007 core_funcs.pyx:63(find_overlapping_leafs_recursive)
 48742104    4.986    0.000    4.986    0.000 core_funcs.pyx:98(overlaps)
   508821    1.593    0.000    2.311    0.000 core_funcs.pyx:33(choose_subtree_least_enlargement)
 11840504    0.739    0.000    0.739    0.000 core_funcs.pyx:517(area)
    20570    0.618    0.000    0.860    0.000 core_funcs.pyx:340(PickSeeds)
   139410    0.573    0.000    0.831    0.000 core_funcs.pyx:376(PickNext)
   100000    0.543    0.000    4.873    0.000 {rbush.core_funcs.insert}
  5794080    0.499    0.000    0.499    0.000 core_funcs.pyx:520(common_boundaries)
        1    0.438    0.438    5.428    5.428 core.py:94(load)
        1    0.333    0.333   26.283   26.283 profiling.py:12(runme)
   100000    0.117    0.000    4.990    0.000 core.py:47(insert)
   529397    0.115    0.000    0.186    0.000 core_funcs.pyx:30(common_boundaries)
   508821    0.097    0.000    0.097    0.000 core_funcs.pyx:260(parent_record)
   668801    0.051    0.000    0.051    0.000 core_funcs.pyx:510(init_Dfloat4)
   101000    0.019    0.000    0.019    0.000 core_funcs.pyx:494(tuple_to_array)
   120564    0.012    0.000    0.012    0.000 core_funcs.pyx:268(is_full)
   100000    0.010    0.000    0.010    0.000 core_funcs.pyx:113(__init__)
        4    0.007    0.002    0.007    0.002 {method 'randn' of 'mtrand.RandomState' objects}
     1000    0.003    0.000   20.508    0.021 core.py:145(search)
        4    0.003    0.001    0.003    0.001 {method 'random_sample' of 'mtrand.RandomState' objects}
     1000    0.003    0.000   20.505    0.021 {rbush.core_funcs.search}
    20577    0.003    0.000    0.003    0.000 core_funcs.pyx:18(__cinit__)
    20576    0.002    0.000    0.002    0.000 core_funcs.pyx:132(__cinit__)
        2    0.002    0.001    0.012    0.006 __init__.py:4(generate_numpy_data)
     1000    0.002    0.000    6.669    0.007 core_funcs.pyx:478(find_overlapping_leafs)
        2    0.002    0.001    0.002    0.001 {built-in method numpy.core.multiarray.array}
        1    0.000    0.000    0.001    0.001 {rbush.core_funcs.create_root}
        1    0.000    0.000   26.284   26.284 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 core_funcs.pyx:150(create_root)
        1    0.000    0.000   26.284   26.284 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.arange}
        2    0.000    0.000    0.014    0.007 __init__.py:13(generate_data_array)
        1    0.000    0.000    0.001    0.001 core.py:20(clear)
        1    0.000    0.000    0.001    0.001 core.py:15(__init__)
       12    0.000    0.000    0.000    0.000 core_funcs.pyx:74(addChild)
        6    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.len}
```

# Let's see what coverage is about

I understand that the line-tracing with the annotation done by `coverage` is helpful to see some sharp edges, let's check.

Now, instead of using `# cython: profile=True` we'll use `# cython: linetrace=True` at the top of our cython module.
Also, we have to add a `.coveragerc` file in our current directory with the content:
```
[run]
plugins = Cython.Coverage
```

Run the profiling script (above) and format that to a HTML for better visualization:
```bash
# cython  --annotate-coverage coverage.xml  rbush/core_funcs.pyx
```

In [3]:
from IPython.display import HTML
from IPython.display import display

display(HTML('core_funcs.html'))

# Line profiling

Line profiling (using `line_profile`) is a bit more complicated...not clear at all to me. But here goes a tentative after the docs:
* https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line
* http://nbviewer.jupyter.org/gist/tillahoffmann/296501acea231cbdf5e7
* http://docs.cython.org/en/latest/src/reference/compilation.html
* http://blog.yclin.me/gsoc/2016/07/23/Cython-IPython/

In [5]:
#Load Robert Kern's line profiler
%load_ext line_profiler
import line_profiler

#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
# from Cython.Compiler.Options import directive_defaults
# directive_defaults['linetrace'] = True
# directive_defaults['binding'] = True

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [8]:
%load_ext Cython

In [13]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1

from rbush.data import generate_data_array
import rbush

def runme():
    data = generate_data_array(100000)
    r = rbush.RBush()
    r.load(data)
    dt = generate_data_array(1000)
    for i in range(len(dt)):
        _ = r.search(*tuple(dt[i]))

runme()

In [21]:
#Print profiling statistics using the `line_profiler` API
profile = line_profiler.LineProfiler(runme)
profile.runcall(runme)
profile.print_stats()

/anaconda/envs/py-rbush/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Could not extract a code object for the object <built-in function runme>
  


Timer unit: 1e-06 s

